# Import necessary libraries

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os, sys

from multiprocessing import Pool
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import *

from sklearn.ensemble import *

from sklearn.naive_bayes import *
from sklearn.linear_model import *
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score

os.chdir("/home/sieu/PycharmProjects/ML-Vincent-Ng/project")

# Training import

In [2]:
column_name = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
df = pd.read_table('train.txt', sep="\s+", usecols=all, names = list(column_name['attr']))

In [3]:
df = df.drop_duplicates()
df = df.dropna()
# Save year
X_year =  df.loc[:,'YEAR']
df.drop(['YEAR'], axis=1)

,B1,B2,B3,C1,C2,C3,C4,C5,C6,C7,...,CT22,CT23,CT24,CT25,CT26,CH1,CH2,CH3,CH4,Class
0,0,0,1,1.260,1.17,0.720,4.590,0.45,0.765,0.540,...,3,2,2,1,3,0,0,0,0,1
1,0,1,0,0.450,0.81,0.000,0.000,0.00,0.855,0.000,...,3,2,1,1,3,0,0,0,0,1
2,0,0,1,0.540,2.88,0.000,0.000,0.00,0.765,0.000,...,3,2,2,1,3,0,0,0,0,1
3,0,0,1,0.810,1.35,0.450,0.000,0.00,0.000,0.720,...,3,2,2,1,3,0,0,0,0,1
4,0,0,1,0.900,1.17,0.765,0.000,0.00,0.630,0.810,...,4,3,2,1,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,1,0,0.450,1.17,0.630,1.215,0.45,0.990,0.540,...,3,2,2,1,3,0,0,0,0,0
11996,0,1,0,5.085,0.90,0.000,0.000,0.00,0.000,0.945,...,4,3,2,1,3,0,0,0,0,1
11997,0,0,1,1.170,1.71,0.630,0.000,0.00,0.720,0.000,...,3,2,1,1,3,0,0,0,0,0
11998,0,0,1,0.810,1.17,0.000,0.000,0.00,0.990,0.810,...,5,3,2,2,4,0,0,0,0,1


In [4]:
# Seperate data by type
y = df["Class"]
X_B = df.loc[:, 'B1':'B3']
X_C = df.loc[:,'C1':'C139']
X_CT = df.loc[:,'CT1':'CT26']
X_H = df.loc[:,'CH1':'CH4']

In [6]:
# Calculate statistics
print(X_B.describe())
print(X_C.describe())
print(X_CT.describe())
print(X_H.describe())
print(X_year.describe())
print(np.unique(df[["CT22"]]))

                 B1            B2            B3
count  11982.000000  11982.000000  11982.000000
mean       0.332916      0.340427      0.326657
std        0.471276      0.473873      0.469010
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        1.000000      1.000000      1.000000
max        1.000000      1.000000      1.000000
                 C1            C2            C3            C4            C5  \
count  11982.000000  11982.000000  11982.000000  11982.000000  11982.000000   
mean       2.087392      9.599654      0.625287      0.287599      0.148434   
std       13.490473     32.454435      1.112775      0.536472      0.405063   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.450000      1.080000      0.000000      0.000000      0.000000   
50%        0.720000      1.350000      0.630000      0.000000      0.000000   
75%        1.17

In [8]:
# Convert to numpy array
#y = y.to_numpy()
#X_B = X_B.to_numpy()
#X_year = X_year.to_numpy()

In [ ]:
# Below are the columns retained by each
# List contains all attributes retained during feature selection: attr_lst
x_c_cols = X_C.columns
x_ct_cols = X_CT.columns
x_h_cols = X_H.columns
# Feature selection based on variance threshold for every type of data
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

# X_C dataframe
sel.feature_names_in_= x_c_cols
X_C = sel.fit_transform(X_C)
x_c_cols=sel.get_feature_names_out(x_c_cols)
attr_lst=x_c_cols
print(x_c_cols)

# X_CT dataframe
sel.feature_names_in_= x_ct_cols
X_CT = sel.fit_transform(X_CT)
x_ct_cols=sel.get_feature_names_out(x_ct_cols)
attr_lst=np.concatenate((attr_lst, x_ct_cols))
print(x_ct_cols)

# X_H dataframe
sel.feature_names_in_= x_h_cols
X_H = sel.fit_transform(X_H)
x_h_cols=sel.get_feature_names_out(x_h_cols)
attr_lst=np.concatenate((attr_lst, x_h_cols))
print(x_h_cols)

In [ ]:
X=np.concatenate((X_C, X_CT, X_H), axis=1)
print(attr_lst)
print(X.shape)

# Feature selection based on Ridge classifier on class
print("BEGIN: Ridge classifier feature selection for C* atttributes")
selX = SequentialFeatureSelector(estimator=RidgeClassifierCV(), n_features_to_select=X.shape[1]//10, direction="forward")
selX.feature_names_in_= attr_lst
X = selX.fit_transform(X, y)
attr_lst = selX.get_feature_names_out(attr_lst)
print(attr_lst)
print("Done Ridge classifier feature selection for C* atttributes")

print('Perform PCA on remaining feature to filter noises')
pca = PCA(svd_solver='full', n_components='mle')
pca.feature_names_in_= attr_lst
X = pca.fit_transform(X)
principal_comp = pca.get_feature_names_out(attr_lst)
# Save models
with open("feat_sel_result.pkl", 'wb') as file:
    pickle.dump((attr_lst, principal_comp), file)
print(X.shape)

In [20]:
# Concat X_B with X to get the final attribute array
X_new=np.concatenate((X_B,X), axis=1)

# GridSearch for best model

In [278]:
params = {'max_iter':np.arange(200,400,20),'max_leaf_nodes':np.arange(200,400, 5), 'l2_regularization':np.arange(0.5,1,0.05), 'learning_rate':np.arange(0.5,1,0.05), 'max_depth':np.arange(2, 40, 2), 'max_bins':np.arange(2, 150, 2)}
#params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = HalvingRandomSearchCV(estimator=HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), 
                                param_distributions=params, factor=2,
                                n_jobs=-1).fit(X_new,y)

In [279]:
# Best model validation
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
arr = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #print(clf.score(X_test, y_test))
    #arr.append(clf.score(X_test, y_test))
    arr.append(search.score(X_test, y_test))
    print(search.score(X_test, y_test))
print("Done")
print("mean acc: ", np.mean(arr))

0.908256880733945
0.890742285237698
0.9181969949916527
0.8989983305509182
0.8948247078464107
0.9006677796327212
0.9048414023372288
0.8973288814691152
0.9106844741235393
0.8939899833055092
Done
mean acc:  0.9018531720228739


In [274]:
# Save models
with open("89_low_depth_leaf_model.pkl", 'wb') as file:
    pickle.dump(search, file)

In [232]:
# Load model
with open("733_model.pkl", 'rb') as file:
    search = pickle.load(file)

In [280]:
search.best_params_

{'max_leaf_nodes': 310,
 'max_iter': 380,
 'max_depth': 38,
 'max_bins': 144,
 'learning_rate': 0.5,
 'l2_regularization': 0.9500000000000004}

# Essemble

In [ ]:
def Ridge_classifiers(data):
    clf = RidgeClassifier(solver="svd")
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def knn_classifiers(data):
    clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def svc_classifiers(data):
    clf = SVC(kernel='rbf', degree=3)
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def BernoulliNB_classifiers(data):
    clf = BernoulliNB()
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def HGB_classifiers(data):
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2])
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)

In [ ]:
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
dataset = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dataset.append((X_train, y_train, X_test, y_test))
    #Testing individual clf
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = RidgeClassifier(solver="svd").fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = BernoulliNB().fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
print("Done")

In [ ]:
assembly = []
vote_weights = []
def append_results(result):
    for clf, score in result:
        assembly.append(clf)
        vote_weights.append(score)
if __name__ == '__main__':
    p = Pool()
    p.map_async(Ridge_classifiers, dataset, callback=append_results)
    #p.map_async(nn_classifiers, dataset, callback=append_results)
    #p.map_async(knn_classifiers, dataset, callback=append_results)
    #p.map_async(svc_classifiers, dataset, callback=append_results)
    p.map_async(BernoulliNB_classifiers, dataset, callback=append_results)
    p.map_async(HGB_classifiers, dataset, callback=append_results)
    p.close()
    p.join()

In [30]:
print(assembly)
print(vote_weights)

[HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2])]
[0.7297748123436196, 0.7272727272727273, 0.7078464106844741, 0.7212020033388982, 0.7170283806343907, 0.7387312186978298, 0.7053422370617696, 0.7036727879799666, 0.7086811352253757, 0.7203672787979967]


In [14]:
clfs = []
for i in range(len(assembly)):
    clfs.append(("Classifier " + str(i), assembly[i]))
eclf = VotingClassifier(estimators=clfs,voting='hard', weights=vote_weights, n_jobs=-1)

In [26]:
cross_val_score(eclf, X_new, y, scoring='accuracy', cv=10, n_jobs=-1)

array([0.69307756, 0.71142619, 0.71869783, 0.70367279, 0.71702838,
       0.71869783, 0.70283806, 0.70367279, 0.68196995, 0.67529215])

# Evaluation 

In [51]:
attr = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
dfX_pred = pd.read_table('sample-test.txt', sep="\s+", usecols=all, names = list(attr['attr'])[:-1])
dfX_pred = dfX_pred.dropna()
dfX_pred

,B1,B2,B3,C1,C2,C3,C4,C5,C6,C7,...,CT21,CT22,CT23,CT24,CT25,CT26,CH1,CH2,CH3,CH4
0,0,1,0,0.450,0.000,0.000,0.00,0.0,0.720,0.00,...,5,3,2,2,2,3,0,0,0,0
1,0,1,0,0.720,1.710,0.810,0.54,0.0,0.630,0.81,...,4,3,2,1,1,3,0,0,0,0
2,0,0,1,0.450,0.675,0.630,0.00,0.0,0.000,0.00,...,5,3,2,2,1,3,1,0,0,0
3,0,0,1,0.675,1.845,1.035,0.72,0.0,1.485,1.35,...,4,3,2,1,1,3,0,0,0,0
4,0,1,0,23.580,1.710,0.000,0.00,0.0,0.630,0.90,...,4,3,2,2,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,1,1.170,1.845,0.675,0.00,0.0,0.720,0.00,...,4,3,2,1,1,3,0,0,0,0
4996,1,0,0,0.540,1.170,1.665,0.81,0.9,0.630,0.63,...,4,3,2,2,1,3,0,0,0,0
4997,0,0,1,0.000,1.170,0.450,0.54,0.0,0.000,0.45,...,5,4,2,2,1,3,0,0,0,0
4998,0,1,0,0.000,1.170,0.630,0.00,0.0,0.450,0.63,...,5,3,2,2,1,3,0,0,0,0


In [52]:
# Seperate prediction data by type
X_pred_B = dfX_pred.loc[:, 'B1':'B3']
X_pred_C = dfX_pred.loc[:,'C1':'C139']
X_pred_CT = dfX_pred.loc[:,'CT1':'CT26']
X_pred_H = dfX_pred.loc[:,'CH1':'CH4']

In [53]:
with open("feat_sel_result.pkl", 'rb') as file:
    attr_lst, principal_comp = pickle.load(file)
list(attr_lst)
list(principal_comp)

['C5', 'C8', 'C17', 'C26', 'C31', 'C59', 'C64', 'C70', 'C95', 'C97', 'CT15']

In [55]:
# List contains all attributes retained during feature selection: attr_lst
# Feature selection based on variance threshold for every type of data

X_pred=pd.concat((X_pred_C, X_pred_CT, X_pred_H), axis=1)
X_pred=X_pred.loc[:,list(attr_lst)]
X_pred

,C5,C8,C17,C26,C31,C59,C64,C70,C95,C97,CT15
0,0.0,0.810,0.0000,4310.3058,1.44,0.3147,0.0000,90.0000,0.0000,0.0000,1
1,0.0,0.540,18.2828,1459.9008,149.85,3.8840,0.0000,947.2592,0.0000,14.3390,1
2,0.0,0.000,64.0811,2898.7020,1.62,0.0000,0.0000,67.0820,0.0000,0.0000,1
3,0.0,0.000,35.4043,2285.9766,5.85,0.8000,241.8677,108.1665,0.0000,1.6250,1
4,0.0,0.450,19.7987,1213.0019,47.16,0.9373,0.0000,67.0820,0.0000,3.5333,1
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.000,23.0812,1625.5714,1.17,0.0000,0.0000,84.8528,0.0000,0.0000,1
4996,0.9,0.000,15.5311,306.0356,2.34,0.2560,13868.5039,360.6804,1.1429,3.1429,1
4997,0.0,0.000,14.0762,74.4680,0.00,0.0000,0.0000,4268.5240,0.0000,4.6500,1
4998,0.0,1.395,58.1604,805.7750,0.00,0.0000,0.0000,84.8528,0.0000,4.6471,1


In [56]:
print('Perform PCA on remaining feature to filter noises')
pca = PCA(svd_solver='full', n_components=len(principal_comp))
X_pred = pca.fit_transform(X_pred)

Perform PCA on remaining feature to filter noises


array([[ 2.51511644e+03, -7.54360101e+00, -6.93350898e+01, ...,
         5.46827705e-02,  5.22954935e-01, -2.12863682e-02],
       [-4.47258370e+02,  2.78840355e+02, -1.56637038e+02, ...,
        -2.10453788e+00, -9.10301763e-02, -7.05761184e-02],
       [ 1.13558671e+03, -3.04690500e+02, -1.14122973e+02, ...,
        -1.66366428e-02, -2.66362819e-01, -6.22930195e-02],
       ...,
       [-2.45070908e+03,  3.26748664e+03, -1.96220424e+02, ...,
         2.43868426e-02, -2.37178809e-01, -7.68232273e-02],
       [-9.19709698e+02, -6.94150197e+02, -1.80671131e+02, ...,
        -3.37196988e-01,  1.04525473e+00, -1.11397336e-02],
       [-6.91343485e+02,  1.45342167e+03, -1.56414618e+02, ...,
        -6.34041972e-01, -3.98264021e-01, -5.61189063e-02]])

In [44]:
# Concat X_B with X to get the final attribute array
X_pred_new=np.concatenate((X_pred_B,X_pred), axis=1)

In [63]:
# Load model then predict and save prediction
with open("91_model.pkl", 'rb') as file:
    search = pickle.load(file)
y_pred = search.predict(X_pred_new)
np.savetxt(fname="prediction.txt", X=y_pred, fmt='%d')